# Bitcoin price prediction

## Imports

In [10]:
#Packages
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
from pandas_datareader import data


from prophet import Prophet


In [24]:
# Bitcoin dataset
btc_data = data.DataReader("BTC-USD",
                          start='2013-04-28',
                          data_source='yahoo')['Adj Close']

btc = pd.DataFrame(btc_data)


In [16]:
btc.head()

,Adj Close
Date,
2014-09-17,457.334015
2014-09-18,424.440002
2014-09-19,394.795990
2014-09-20,408.903992
2014-09-21,398.821014


In [19]:
# Visualitation 
px.area(btc,
        x=btc.index,
        y='Adj Close')

In [26]:
# Taking the logs
btc['log_price'] = np.log(btc["Adj Close"])

px.area(btc,
        x=btc.index,
        y='log_price')


In [41]:
# Renaming the columns for Prophet
prophet_df = pd.DataFrame(btc['log_price'])
prophet_df['ds'] = btc.index
prophet_df.rename(columns={'log_price':'y'}, inplace=True)
prophet_df.head()

,y,ds
Date,,
2014-09-17,6.125414,2014-09-17
2014-09-18,6.050771,2014-09-18
2014-09-19,5.978369,2014-09-19
2014-09-20,6.013480,2014-09-20
2014-09-21,5.988513,2014-09-21


In [43]:
priors = [0.0015, 0.015]
prophets, labels = [], []
for prior in priors:
    prophet = Prophet(changepoint_prior_scale=prior)
    prophet.fit(prophet_df)
 
    prophets.append(prophet)
    labels.append(r"CP Prior = " + str(prior))

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [46]:
forecasts = []
for prophet in prophets:
    forecast = prophet.make_future_dataframe(periods=365 * 2,                                                    freq="D")
    forecast = prophet.predict(forecast)
 
    forecast = forecast.rename(columns={"ds" : str(priors[prophets.index(prophet)]) + "_ds"})
    forecasts.append(forecast)

In [45]:
labels

['CP Prior = 0.0015', 'CP Prior = 0.015']

In [22]:
model = Prophet()
model.add_country_holidays(country_name='US')
model.fit(btc)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [57]:
import itertools
import numpy as np
import pandas as pd
from prophet.diagnostics import cross_validation, performance_metrics


param_grid= {
    'changepoint_prior_scale':[0.001, 0.01, 0.1, 0,2], 
    'seasonality_prior_scale':[0.01, 0.1, 1.0, 5],
}

all_params= [dict(zip(param_grid.keys(), values))
             for values in itertools.product(*param_grid.values())]
rmses= []

for params in all_params:
    m= Prophet(**params).fit(prophet_df)
    df_cv= cross_validation(m, horizon='360 days', parallel="processes")
    df_p= performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

tuning_results= pd.DataFrame(all_params)
tuning_results['rmses']= rmses


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 8 forecasts with cutoffs between 2017-09-26 00:00:00 and 2021-03-09 00:00:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x0000021E1A384790>
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 8 forecasts with cutoffs between 2017-09-26 00:00:00 and 2021-03-09 00:00:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x0000021E1A263EB0>
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 8 forecasts with cutoffs between 2017-09-26 00:00:00 and 2021-03-09 00:00:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x0000021E092C74F0>
INFO:prophet:Disabling daily seasonality. Run prophet with

In [55]:
all_params

[{'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 0.01},
 {'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 0.1},
 {'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 1.0},
 {'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 5},
 {'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 10},
 {'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 10.0},
 {'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.01},
 {'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.1},
 {'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 1.0},
 {'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 5},
 {'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 10},
 {'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 10.0},
 {'changepoint_prior_scale': 0.1, 'seasonality_prior_scale': 0.01},
 {'changepoint_prior_scale': 0.1, 'seasonality_prior_scale': 0.1},
 {'changepoint_prior_scale': 0.1, 'seasonalit